# Introduction to backtesting trading strategies using `zipline`

Learn how to build and backtest trading strategies using `zipline`

## Introduction

In this article, I would like to continue the series on quantitative finance. In the [first part](https://towardsdatascience.com/introduction-to-quantitative-finance-part-i-stylised-facts-of-asset-returns-5190581e40ea), I described the stylized facts of asset returns. Now I would like to introduce the concept of backtesting trading strategies and how to do it using existing frameworks in Python.

## What is backtesting?

Let's start with a trading strategy. It can be defined as a method of buying and/or selling assets in markets that is based on predefined rules. These rules can be based on, for example, technical analysis or machine learning models.

Backtesting is basically evaluating the performance of a trading strategy on historical data - if we used a given strategy on a set of assets in the past, how well/bad would it have performed. Of course, there is no guarantee that past performance is indicative of the future one, but we can still investigate!

There are a few available frameworks for backtesting in Python, in this article, I decided to use `zipline`.

## Why zipline?

Some of the nice features offered by the `zipline` environment include:
* ease of use - there is a clear structure of how to build a backtest and what outcome we can expect, so the majority of the time can be spent on developing state-of-the-art trading strategies :)
* realistic - includes transaction costs, slippage, order delays, etc.
* stream-based - processes each event individually, thus avoids look-ahead bias
* it comes with many easily-accessible statistical measures, such as moving average, linear regression, etc. - no need to code them from scratch
* integration with PyData ecosystem - `zipline` uses Pandas DataFrames for storing input data, as well as performance metrics
* it is easy to integrate other libraries, such as `matplotlib`, `scipy`, `statsmodels` and `sklearn` into the workflow of building and evaluating strategies
* developed and updated by Quantopian which provides a web-interface for `zipline`, historical data and even live-trading capabilities

I believe these arguments speak for themselves. Let's start coding!

## Setting up the virtual environment using conda

The most convenient way to install `zipline` is to use a virtual environment. In this article, I use `conda` to do so. I create a new environment with Python 3.5 (I experienced issues with using 3.6 or 3.7) and then install `zipline`. You can also `pip install` it.

In [1]:
# create new virtual environment
#conda create -n env_zipline python=3.5

# activate it
#conda activate env_zipline

# install zipline
#conda install -c Quantopian zipline

For everything to be working properly you should also install `jupyter` and other packages used in this article (see the `watermark` printout below).

## Importing libraries

First, we need to load IPython extensions using the `%load_ext` magic.

In [1]:
import sys

print(sys.version)
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)

3.6.6 (default, Oct 16 2018, 07:17:20) 
[GCC 6.3.0 20170516]
1.19.5
0.25.3


In [2]:
%load_ext watermark

In [3]:
%load_ext zipline

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Then we import the rest of the libraries:

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import zipline
from trading_calendars import get_calendar
import yfinance as yf
import warnings

plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 200
warnings.simplefilter(action='ignore', category=FutureWarning)

Below you can see the list of libraries used in this article, together with their versions.

In [6]:
%watermark --iversions

numpy    1.19.5
zipline  1.6.1+6.gacc6dde7.dirty
pandas   0.25.3
yfinance 0.1.60



## Import custom data

`zipline` comes ready with data downloaded from Quandl (the WIKI database). You can always inspect the already ingested data by running: 

In [7]:
!zipline bundles

alpaca_api <no ingestions>
alpha_vantage <no ingestions>
csvdir <no ingestions>
quandl <no ingestions>
quantopian-quandl <no ingestions>


The issue with this approach is that in mid 2018 the data was discontinued, so there is no data for the last year. To overcome this, I show how to manually ingest data from any source. To do so I use the `yahoofinancials` library. In order to be loaded into `zipline`, the data must be in a CSV file and in a predefined format - like the one on the preview of the DataFrame.

In [22]:
def get_data(start, end, codelist):
  data = yf.download(codelist, start, end)
  # display start date of each stock
#  for i in codelist:
#    display(data[i].dropna().head(1))

  data=data.interpolate('time') # 欠損データを補間
  #data_drop=data.dropna() # 欠損データを落とす    

  return data# Adjusted Close (配当込み，分割調整値) の列を抽出

In [23]:
ticker = ['AAPL']
date_start = '2017-01-01' #@param {type:"date"}
start = datetime.datetime.strptime(date_start, '%Y-%m-%d')
end = datetime.date.today()

df = get_data(start, end, ticker)
display(df)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,28.950001,29.082500,28.690001,29.037500,27.413372,115127600
2017-01-04,28.962500,29.127501,28.937500,29.004999,27.382690,84472400
2017-01-05,28.980000,29.215000,28.952499,29.152500,27.521944,88774400
2017-01-06,29.195000,29.540001,29.117500,29.477501,27.828764,127007600
2017-01-09,29.487499,29.857500,29.485001,29.747499,28.083660,134247600
...,...,...,...,...,...,...
2021-06-28,133.410004,135.250000,133.350006,134.779999,134.779999,62111300
2021-06-29,134.800003,136.490005,134.350006,136.330002,136.330002,64556100
2021-06-30,136.169998,137.410004,135.869995,136.960007,136.960007,63261400


In [24]:
df = pd.DataFrame(df[ticker[0]]['prices']).drop(['date'], axis=1).rename(columns={'formatted_date':'date'})
df = df[['date','open','high','low','close','volume']]
df['dividend'] = 0
df['split'] = 1

df.head()

KeyError: 'AAPL'

We then need to save the data as a CSV file in a folder called 'daily' (or another folder of your choice).

In [ ]:
df.to_csv('daily/aapl.csv', header=True, index=False)

In the next step, we need to modify the `extension.py` file located in the zipline directory. After the installation of `zipline` it is empty and we need to add the following:

In [ ]:
import pandas as pd

from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities

start_session = pd.Timestamp('2017-01-03', tz='utc')
end_session = pd.Timestamp('2019-06-28', tz='utc')

# register the bundle 
register(
    'apple-prices-2017-2019', # name we select for the bundle
    csvdir_equities(
        ['daily'], # name of the directory as specified above (named after data frequency)
        '/path/to/directory/with/csv', # path to directory containing the data
    ),
    calendar_name='NYSE',  # US equities
    start_session=start_session,
    end_session=end_session
)

We can also define and provide a custom calendar to the data-ingesting script - for example when working with European stocks. For details on how to do it please look at the [documentation](https://www.zipline.io/trading-calendars.html).

In contrast to the data downloading function, we need to pass the exact range of dates of the downloaded data. In this example, we start with `2017-01-03`, as this is the first day for which we have pricing data. 

Lastly, we run the following command:

In [ ]:
!zipline ingest -b apple-prices-2017-2019

We can verify that the bundle was successfully ingested:

In [ ]:
!zipline bundles

There is a [known issue](https://github.com/quantopian/zipline/issues/2480) with downloading the benchmark data, so - for now - we stick to historical data in the default bundle. However, you now know how to ingest data using a custom csv file.

## Buy And Hold Strategy

We start with the most basic strategy - Buy and Hold. The idea is that we buy a certain asset and do not do anything for the entire duration of the investment horizon. This simple strategy can also be considered a benchmark for more advanced ones - because there is no point in using a very complex strategy that generates less money (for example due to transaction costs) than buying and doing nothing.

In this example, we consider Apple's stock and select years 2016–2017 as the duration of the backtest. We start with a capital of 1050 USD. I selected this number as I know how much more or less we need to have for the initial purchase and I like to keep this number as small as possible because we are only buying 10 shares, so no need for a starting balance of a couple of thousands. We assume the default transaction costs (0.001$ per share, without a minimum cost per trade).

There are two approaches to using `zipline` - using the command line or Jupyter Notebook. To use the latter we have to write the algorithm within a Notebook cell and indicate that `zipline` is supposed to run it. This is done via the `%%zipline` IPython magic command. This magic takes the same arguments as the CLI mentioned above. 

Also one important thing, all imports required for the algorithm to run (such as `numpy`, `sklearn`, etc.) must be specified in the algorithm cell, even if they were previously imported elsewhere.

In [ ]:
%%zipline --start 2016-1-1 --end 2017-12-31 --capital-base 1050.0 -o buy_and_hold.pkl

# imports
from zipline.api import order, symbol, record

# parameters
selected_stock = 'AAPL'
n_stocks_to_buy = 10

def initialize(context):
    context.has_ordered = False  

def handle_data(context, data):
    # record price for further inspection
    record(price=data.current(symbol(selected_stock), 'price'))
    
    # trading logic
    if not context.has_ordered:
        # placing order, negative number for sale/short
        order(symbol(selected_stock), n_stocks_to_buy)
        # setting up a flag for holding a position
        context.has_ordered = True

Congrats, we have written our first backtest. So what actually happened?

Each `zipline` algorithm contains (at least) two functions we have to define:
* `initialize(context)` 
* `handle_data(context, data)`

Before the algorithm starts, the `initialize()` function is called and a `context` variable is passed. `context` is a global variable in which we can store additional variables we need to access from one iteration of the algorithm to the next.

After the initialization of the algorithm, the `handle_data()` function is called once for each event. At every call, it passes the same `context` variable and an event frame called `data`. It contains the current trading bar with open, high, low, and close (OHLC) prices together with the volume.

We create an order by using `order(asset, number_of_units)`, where we specify what to buy and how many shares/units. A positive number indicates buying that many shares, 0 means selling everything we have, and a negative number is used for short-selling. Another useful type of order is `order_target`, which orders as many shares as needed to achieve the desired number in the portfolio.

In our Buy and Hold strategy, we check if we have already placed an order. If not, we order a given amount of shares and then do nothing for the rest of the backtest.

Let's analyze the performance of the strategy. First, we need to load the performance DataFrame from the pickle file.

In [ ]:
# read the performance summary dataframe
buy_and_hold_results = pd.read_pickle('buy_and_hold.pkl')

And now we can plot some of the stored metrics:

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=[16, 9])

# portfolio value
buy_and_hold_results.portfolio_value.plot(ax=ax[0])
ax[0].set_ylabel('portfolio value in $')

# asset
buy_and_hold_results.price.plot(ax=ax[1])
ax[1].set_ylabel('price in $')

# mark transactions
perf_trans = buy_and_hold_results.loc[[t != [] for t in buy_and_hold_results.transactions]]
buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
sells = perf_trans.loc[[t[0]['amount'] < 0 for t in perf_trans.transactions]]
ax[1].plot(buys.index, buy_and_hold_results.price.loc[buys.index], '^', markersize=10, color='g', label='buy')
ax[1].plot(sells.index, buy_and_hold_results.price.loc[sells.index], 'v', markersize=10, color='r', label='sell')

# daily returns
buy_and_hold_results.returns.plot(ax=ax[2])
ax[2].set_ylabel('daily returns')

fig.suptitle('Buy and Hold Strategy - Apple', fontsize=16)
plt.legend()
plt.show()

print('Final portfolio value (including cash): {}$'.format(np.round(buy_and_hold_results.portfolio_value[-1], 2)))

From the first look we see that the portfolio generated money over the investment horizon and was very much following the price of Apple (what makes sense as it is the only asset in the portfolio).

To view the transactions we need to transform the `transactions` column from the performance `DataFrame`.

In [ ]:
pd.DataFrame.from_records([x[0] for x in buy_and_hold_results.transactions.values if x != []])

By inspecting the columns of the performance `DataFrame` we can see all the available metrics.

In [ ]:
buy_and_hold_results.columns

Some of the noteworthy ones:
* starting/ending cash - inspecting the cash holding on a given day
* starting/ending value - inspecting the assets; value on a given day
* orders - used for inspecting orders; there are different events for creating an order when the trading strategy generates a signal, and a separate one when it is actually executed on the next trading day
* pnl - daily profit and loss

## Simple Moving Average Strategy

The second strategy we consider is based on the simple moving average (SMA). The 'mechanics' of the strategy can be summarized by the following:

* when the price crosses the 20-day SMA upwards - buy x shares
* when the price crosses the 20-day SMA downwards - sell the shares
* we can only have a maximum of x shares at any given time
* there is no short-selling in the strategy (though it can be easily implemented)

The remaining components of the backtest like the considered asset, investment horizon or the starting capital is the same as in the Buy and Hold example.

In [ ]:
%%zipline --start 2016-1-1 --end 2017-12-31 --capital-base 1050.0 -o sma_strategy.pkl

# imports 
from zipline.api import order_target, record, symbol
from zipline.finance import commission
import matplotlib.pyplot as plt
import numpy as np

# parameters 
ma_periods = 20
selected_stock = 'AAPL'
n_stocks_to_buy = 10

def initialize(context):
    context.time = 0
    context.asset = symbol(selected_stock)
    # 1. manually setting the commission
    context.set_commission(commission.PerShare(cost=0.001, min_trade_cost=0))

def handle_data(context, data):
    # 2. warm-up period
    context.time += 1
    if context.time < ma_periods:
        return

    # 3. access price history
    price_history = data.history(context.asset, fields="price", bar_count=ma_periods, frequency="1d")
 
    # 4. calculate moving averages
    ma = price_history.mean()
    
    # 5. trading logic
    
    # cross up
    if (price_history[-2] < ma) & (price_history[-1] > ma):
        order_target(context.asset, n_stocks_to_buy)
    # cross down
    elif (price_history[-2] > ma) & (price_history[-1] < ma):
        order_target(context.asset, 0)

    # save values for later inspection
    record(price=data.current(context.asset, 'price'),
           moving_average=ma)
    
# 6. analyze block
def analyze(context, perf):
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=[16, 9])

    # portfolio value
    perf.portfolio_value.plot(ax=ax[0])
    ax[0].set_ylabel('portfolio value in $')
    
    # asset
    perf[['price', 'moving_average']].plot(ax=ax[1])
    ax[1].set_ylabel('price in $')
    
    # mark transactions
    perf_trans = perf.loc[[t != [] for t in perf.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[[t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax[1].plot(buys.index, perf.price.loc[buys.index], '^', markersize=10, color='g', label='buy')
    ax[1].plot(sells.index, perf.price.loc[sells.index], 'v', markersize=10, color='r', label='sell')
    ax[1].legend()
    
    # daily returns
    perf.returns.plot(ax=ax[2])
    ax[2].set_ylabel('daily returns')

    fig.suptitle('Simple Moving Average Strategy - Apple', fontsize=16)
    plt.legend()
    plt.show()
    
    print('Final portfolio value (including cash): {}$'.format(np.round(perf.portfolio_value[-1], 2)))


The code for this algorithm is a little bit more complex, but we will cover all the new aspects of the code. For simplicity, I marked the points of reference in the code snippet above and will refer to them by number below.

1. I show how to manually set the commission. In this case, I use the default value for comparison's sake.
2. The "warm-up period" - this is a trick used in order to make sure that the algorithm has enough days to calculate the moving average. If we are using multiple metrics with different window lengths, we should always take the longest one for the warm-up. 
3. We access the historical (and current) data-points by using `data.history`. In this example, we access the last 20 days. The data (in case of a single asset) is stored as a `pandas.Series`, indexed by time.
4. The SMA is a very basic measure, so for calculation, I simply take the mean of the previously accessed data.
5. I encapsulate the logic of the trading strategy in an `if` statement. To access the current and previous data-points I use `price_history[-2]` and `price_history[-1]`, respectively. To see if there was a crossover, I compare the current and previous prices to the MA and determine which case I am dealing with (buy/sell signal). In the case where there is no signal, the algorithm does nothing.
6. You can use the `analyze(context, perf)` statement to carry out extra analysis (like plotting) when the backtest is finished. The `perf` object is simply the performance `DataFrame` we also store in a pickle file. But when used withing the algorithm, we should refer to it as `perf` and there is no need for loading it.

As compared to the Buy and Hold strategy, you might have noticed the periods where the portfolio value is flat. That is because when we sell the asset (and before buying again), we only hold cash.

In our case, the Simple Moving Average strategy outperformed the Buy and Hold one. The ending worth of the portfolio (including cash) is 1784.12 USD for the SMA strategy, while it is 1714.68 USD in case of the simpler one.

## Conclusions

In this article, I have shown how to use the `zipline` framework to carry out the backtesting of trading strategies. Once you get familiar with the library, it is easy to test out different strategies. In a future article, I will cover using more advanced trading strategies based on technical analysis. 

As always, any constructive feedback is welcome. You can reach out to me on [Twitter](https://twitter.com/erykml1) or in the comments. You can find the code used for this article on my [GitHub]().